## Wage Trust Data Analysis Workflow Demo

### Reference data cleansing steps: completness, accuracy
1. check missing talbes
2. convert to standard column_name
3. cheking missing key columns
4. check Null value (null values in not-null columns) 
5. convert to predefined datatype
6. check missing values (emp, date)
7. check outliers (setting range when create table)
8. check duplications
9. insert assumptions *

### 1. Dump all data from '/Data/' (For demo purpose)
    - Assume only CSV file
    - Assume there is no JSON columns
    - Dump all CSVs under RAW_DATA_DIR
#### 1.1 Dump support data
#### 1.2 Dump raw data

### 2. import libraries, set data path, create db object

In [27]:
%reload_ext autoreload
%autoreload
import glob
import uuid
import pandas as pd
from _db_data import DBData
import datetime as dt
import time


RAW_DATA_DIR = '../Data/'
db_obj = DBData()

### 3. Data inspection

#### 3.1 Collect data from db

In [28]:
df_ta = db_obj.retrieve_data('raw_tabcorp_data')

#### 3.2 Inspection

In [29]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1957 entries, 0 to 1956
Data columns (total 13 columns):
emp_id                1867 non-null float64
emp_name              1957 non-null object
shift_start_date      1957 non-null object
shift_end_date        1957 non-null object
shift_start_time      1957 non-null object
shift_end_time        1957 non-null object
work_time_tabcorp     1867 non-null float64
break_time_tabcorp    1650 non-null float64
roster_start_date     1957 non-null object
roster_start          1957 non-null object
roster_end            1957 non-null object
work_segment          1957 non-null object
general_segment       1957 non-null object
dtypes: float64(3), object(10)
memory usage: 198.9+ KB


In [30]:
#check emp_id is null
df_ta[df_ta['emp_id'].isnull()].head(10)

,emp_id,emp_name,shift_start_date,shift_end_date,shift_start_time,shift_end_time,work_time_tabcorp,break_time_tabcorp,roster_start_date,roster_start,roster_end,work_segment,general_segment
25,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-05-18,2019-05-18 15:15:00.000,nan,TL Assist,Sick
26,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-05-21,nan,nan,nan,Memo
37,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-05-27,2019-05-27 13:00:00.000,nan,TL Assist,Sick
45,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-05-31,2019-05-31 08:30:00.000,nan,nan,Day Off
46,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-01,2019-06-01 08:30:00.000,nan,nan,Day Off
47,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-02,2019-06-02 08:30:00.000,nan,nan,Day Off
48,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-03,2019-06-03 08:30:00.000,nan,nan,Day Off
49,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-04,2019-06-04 08:30:00.000,nan,nan,Day Off
50,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-05,2019-06-05 08:30:00.000,nan,nan,Day Off
51,NaN,Belinda Parker,nan,nan,nan,nan,NaN,NaN,2019-06-06,2019-06-06 08:30:00.000,nan,nan,Day Off


#### 3.3 Cleansing

In [31]:
#remove emp_is is null and filter columns we need
df_ta = df_ta[df_ta['emp_id'].notnull()]
df_ta = df_ta.filter(['emp_id','shift_start_time', 'shift_end_time'], axis=1)
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1867 entries, 0 to 1956
Data columns (total 3 columns):
emp_id              1867 non-null float64
shift_start_time    1867 non-null object
shift_end_time      1867 non-null object
dtypes: float64(1), object(2)
memory usage: 58.3+ KB


In [32]:
#convert emp_id to string
df_ta['emp_id'] = df_ta['emp_id'].astype('int').astype('str')

#mapping the columns you want to keep
df_ta.rename(columns={'shift_start_time': 'shift_start',
                      'shift_end_time': 'shift_end'}, inplace=True)

# list of columns we need to reformat to datetime
times = ['shift_start','shift_end']

# Reformatting listed columns to datetime
for x in times:
    df_ta[x] = pd.to_datetime(df_ta[x], yearfirst=True)

# Remove duplicated rows
df_ta = df_ta.drop_duplicates()

#save df_ta to cln_shift
db_obj.insert_data(df_ta, 'cln_shift')

In [33]:
df_ta['emp_id']

0       400670
1       400670
2       400670
3       400670
4       400670
         ...  
1952    400422
1953    400422
1954    400422
1955    400422
1956    400422
Name: emp_id, Length: 1867, dtype: object

### 4. Call RDA API

In [34]:
import pandas as pd
import requests
import json

In [35]:
res = requests.post('http://127.0.0.1:8080/rda_working_pattern')
res_df_6 = pd.DataFrame(res.json())
res_df_6.head()

,am,day_of_week,emp_id,emp_shift_date_am_pm,org_id,position_id,shift_date,shift_desc,shift_end,shift_len_mins,shift_overview,shift_start,shift_type
0,am,Sat,400670,400670_2019-05-04_am,None,None,2019-05-04,None,"Sat, 04 May 2019 10:10:12 GMT",99,Sat_am,"Sat, 04 May 2019 08:30:27 GMT",None
1,am,Sat,400670,400670_2019-05-04_am,None,None,2019-05-04,None,"Sat, 04 May 2019 12:32:34 GMT",128,Sat_am,"Sat, 04 May 2019 10:23:43 GMT",None
2,pm,Sat,400670,400670_2019-05-04_pm,None,None,2019-05-04,None,"Sat, 04 May 2019 16:34:05 GMT",211,Sat_pm,"Sat, 04 May 2019 13:02:53 GMT",None
3,pm,Mon,400670,400670_2019-05-06_pm,None,None,2019-05-06,None,"Mon, 06 May 2019 19:34:37 GMT",258,Mon_pm,"Mon, 06 May 2019 15:15:50 GMT",None
4,pm,Mon,400670,400670_2019-05-06_pm,None,None,2019-05-06,None,"Mon, 06 May 2019 23:20:57 GMT",196,Mon_pm,"Mon, 06 May 2019 20:04:44 GMT",None


In [36]:
res = requests.post('http://127.0.0.1:8080/rda_engagement_period')
res_df_8 = pd.DataFrame(res.json())
res_df_8.head()

,daily_ave_hr,emp_id,fortnightly_ave_hr,weekly_ave_hr
0,7.42,400670,41.53,25.96
1,6.89,506114,70.31,34.02
2,6.70,401991,72.30,34.90
3,6.96,400532,32.80,16.40
4,6.95,508520,74.01,33.42


Test to Bring through all other tests

In [37]:
# Test 7
res = requests.post('http://127.0.0.1:8080/rda_test_7')
res_df_7 = pd.DataFrame(res.json())
res_df_7.head()

,break_dur,break_flag,break_not_taken,emp_id,max_shift_end,min_shift_start,next_emp_id,next_shift_date,next_shift_flag,next_shift_start_time,org_id,position_id,shift_date,shift_desc,shift_dur,shift_end,shift_start,shift_type
0,0,1,1,400422,"Wed, 01 May 2019 15:35:04 GMT","Wed, 01 May 2019 08:30:21 GMT",400422,2019-05-01,1,"Wed, 01 May 2019 09:14:34 GMT",None,None,2019-05-01,None,7.078611,"Wed, 01 May 2019 09:14:34 GMT","Wed, 01 May 2019 08:30:21 GMT",None
1,33,1,0,400422,"Wed, 01 May 2019 15:35:04 GMT","Wed, 01 May 2019 08:30:21 GMT",400422,2019-05-01,1,"Wed, 01 May 2019 14:37:29 GMT",None,None,2019-05-01,None,7.078611,"Wed, 01 May 2019 14:03:51 GMT","Wed, 01 May 2019 09:14:34 GMT",None
2,0,1,0,400422,"Wed, 01 May 2019 15:35:04 GMT","Wed, 01 May 2019 08:30:21 GMT",400422,2019-05-02,0,"Thu, 02 May 2019 09:00:52 GMT",None,None,2019-05-01,None,7.078611,"Wed, 01 May 2019 15:35:04 GMT","Wed, 01 May 2019 14:37:29 GMT",None
3,31,1,0,400422,"Thu, 02 May 2019 17:00:03 GMT","Thu, 02 May 2019 09:00:52 GMT",400422,2019-05-02,1,"Thu, 02 May 2019 14:03:35 GMT",None,None,2019-05-02,None,7.986389,"Thu, 02 May 2019 13:32:20 GMT","Thu, 02 May 2019 09:00:52 GMT",None
4,0,1,0,400422,"Thu, 02 May 2019 17:00:03 GMT","Thu, 02 May 2019 09:00:52 GMT",400422,2019-05-03,0,"Fri, 03 May 2019 09:15:17 GMT",None,None,2019-05-02,None,7.986389,"Thu, 02 May 2019 17:00:03 GMT","Thu, 02 May 2019 14:03:35 GMT",None


In [38]:
# Test 9
res = requests.post('http://127.0.0.1:8080/rda_test_9')
res_df_9 = pd.DataFrame(res.json())
res_df_9.head()

,emp_id,max_shift_end,min_gap_not_taken,min_shift_start,next_emp_id,next_shift_date,next_shift_start,shift_date,shift_gap
0,400422,"Wed, 01 May 2019 15:35:04 GMT",0,"Wed, 01 May 2019 08:30:21 GMT",400422,2019-05-02,"Thu, 02 May 2019 09:00:52 GMT",2019-05-01,17.430000
1,400422,"Thu, 02 May 2019 17:00:03 GMT",0,"Thu, 02 May 2019 09:00:52 GMT",400422,2019-05-03,"Fri, 03 May 2019 09:15:17 GMT",2019-05-02,16.253889
2,400422,"Fri, 03 May 2019 17:17:20 GMT",0,"Fri, 03 May 2019 09:15:17 GMT",400422,2019-05-04,"Sat, 04 May 2019 10:30:29 GMT",2019-05-03,17.219167
3,400422,"Sat, 04 May 2019 18:33:46 GMT",0,"Sat, 04 May 2019 10:30:29 GMT",400422,2019-05-07,"Tue, 07 May 2019 08:31:30 GMT",2019-05-04,61.962222
4,400422,"Tue, 07 May 2019 17:50:49 GMT",0,"Tue, 07 May 2019 08:31:30 GMT",400422,2019-05-09,"Thu, 09 May 2019 08:30:14 GMT",2019-05-07,38.656944


In [39]:
# Test 10
res = requests.post('http://127.0.0.1:8080/rda_test_10')
res_df_10 = pd.DataFrame(res.json())
res_df_10.head()

,count_of_consecutive_days_worked,emp_id
0,2,400422
1,2,400422
2,2,400422
3,2,400422
4,3,400422


### 5. Save results to database

In [40]:
#save df_ta to cln_shift
db_obj.insert_data(res_df_6, 'results_test_6')

In [41]:
#save df_ta to cln_shift
db_obj.insert_data(res_df_7, 'results_test_7')

In [42]:
db_obj.insert_data(res_df_8, 'results_test_8')

In [43]:
db_obj.insert_data(res_df_9, 'results_test_9')

In [44]:
db_obj.insert_data(res_df_10, 'results_test_10')

### 5. Save results to output folder (version control issue)

In [45]:
#Save results as the input of tableau
res_df_6.to_csv('../Data/output/text_6.csv',index=False)
res_df_7.to_csv('../Data/output//text_7.csv',index=False)
res_df_8.to_csv('../Data/output//text_8.csv',index=False)
res_df_9.to_csv('../Data/output//text_9.csv',index=False)
res_df_10.to_csv('../Data/output//text_10.csv',index=False)